# Detecting EMA cross traps

### Import Library

In [23]:
import numpy as np
import pandas as pd
import numpy as np
import pandas_ta as ta
import seaborn as sns

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 120
import warnings
warnings.filterwarnings('ignore')

### Load Price Data

In [24]:
import os
from pathlib import Path
notebook_path = os.getcwd()
current_dir = Path(notebook_path)
csv_file = str(current_dir) + '/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    print('remote')
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [25]:
data = dataset.copy()

In [26]:
data["ema_fast"] = ta.ema(data["Close"], length=20)
data["ema_low"] = ta.ema(data["Close"], length=250)
data["ema_cross"] = ((data["ema_fast"] > data["ema_low"]) & (data["ema_fast"].shift(1) <= data["ema_low"].shift(1)) | (data["ema_fast"] < data["ema_low"]) & (data["ema_fast"].shift(1) >= data["ema_low"].shift(1)))

## Preparing data for detecting TRAP

In [27]:
def is_trap(r):
    trap = ''
    if r['ema_cross'] == True:
        if r['ema_fast'] > r['ema_low']:
            # Cross up
            if r['min_low_1dlater'] < r['Close'] - 3.5:
                trap = 1
            else:
                trap = 0
        else:
            # Cross down
            if r['max_high_1dlater'] > r['Close'] + 3.5:
                trap = 1
            else:
                trap = 0
    return trap

In [28]:
data['max_high_1dlater'] = data['High'].shift(-51).rolling(51).max()
data['min_low_1dlater'] = data['Low'].shift(-51).rolling(51).min()
data['trap'] = data.apply(lambda r: is_trap(r), axis=1)

In [29]:
# data.dropna(inplace=True)

In [30]:
data

,Open,High,Low,Close,Volume,ema_fast,ema_low,ema_cross,max_high_1dlater,min_low_1dlater,trap
Date,,,,,,,,,,,
2018-08-13 09:00:00,943.5,943.6,942.9,943.1,1812,NaN,NaN,False,NaN,NaN,
2018-08-13 09:05:00,943.1,943.5,942.9,943.3,1323,NaN,NaN,False,NaN,NaN,
2018-08-13 09:10:00,943.2,943.3,942.6,943.1,1207,NaN,NaN,False,NaN,NaN,
2018-08-13 09:15:00,943.1,943.1,942.3,942.6,1196,NaN,NaN,False,NaN,NaN,
2018-08-13 09:20:00,942.6,943.7,942.4,943.7,1765,NaN,NaN,False,NaN,NaN,
...,...,...,...,...,...,...,...,...,...,...,...
2025-02-14 14:15:00,1343.0,1343.0,1340.3,1341.3,7141,1343.885390,1337.889347,False,NaN,NaN,
2025-02-14 14:20:00,1340.9,1341.9,1340.5,1341.4,4593,1343.648686,1337.917321,False,NaN,NaN,
2025-02-14 14:25:00,1341.1,1342.5,1340.7,1342.5,4207,1343.539287,1337.953836,False,NaN,NaN,


In [31]:
data[data.trap==1]

,Open,High,Low,Close,Volume,ema_fast,ema_low,ema_cross,max_high_1dlater,min_low_1dlater,trap
Date,,,,,,,,,,,
2018-08-30 10:15:00,961.7,961.7,961.0,961.0,1027,962.030275,962.060930,True,970.8,960.6,1
2018-09-17 14:05:00,953.6,954.1,953.2,954.0,2008,953.569207,953.541030,True,955.2,947.0,1
2018-09-17 14:15:00,953.8,953.8,950.4,950.5,5596,953.314022,953.520427,True,957.4,947.0,1
2018-10-02 14:20:00,982.9,982.9,981.2,981.4,1767,984.866050,984.885325,True,987.2,981.7,1
2018-10-03 14:05:00,983.1,983.5,982.8,983.5,2307,984.939624,984.999560,True,988.8,982.9,1
...,...,...,...,...,...,...,...,...,...,...,...
2025-01-16 13:10:00,1300.1,1301.0,1299.7,1300.5,1744,1302.196875,1302.289836,True,1312.7,1295.1,1
2025-02-03 09:55:00,1319.7,1319.8,1318.7,1319.0,2145,1328.814675,1329.317313,True,1327.7,1313.5,1
2025-02-10 09:50:00,1327.5,1328.0,1326.8,1327.9,2759,1334.554459,1335.013120,True,1337.6,1326.8,1


In [32]:
data[data.trap==0]

,Open,High,Low,Close,Volume,ema_fast,ema_low,ema_cross,max_high_1dlater,min_low_1dlater,trap
Date,,,,,,,,,,,
2018-08-21 14:00:00,950.4,951.8,950.2,950.8,2849,948.814559,948.784204,True,960.6,950.3,0
2018-08-30 11:20:00,962.6,963.5,962.5,963.3,1514,962.040668,962.037060,True,973.3,963.0,0
2018-09-04 10:35:00,963.1,963.9,963.1,963.8,1356,964.749051,964.787876,True,965.8,950.7,0
2018-09-11 13:40:00,945.9,947.1,945.9,947.0,1645,944.820616,944.611729,True,962.8,944.6,0
2018-09-17 13:50:00,952.9,953.3,952.3,952.8,1530,953.493137,953.535510,True,954.9,947.0,0
...,...,...,...,...,...,...,...,...,...,...,...
2024-12-05 11:30:00,1313.0,1313.0,1312.8,1312.8,179,1308.759051,1308.758093,True,1348.9,1312.6,0
2024-12-25 09:00:00,1333.6,1334.4,1332.7,1332.7,7620,1326.195300,1325.578251,True,1355.6,1332.0,0
2025-01-17 09:15:00,1311.4,1311.4,1308.6,1309.1,6481,1302.375442,1302.037473,True,1323.3,1307.7,0


## Prepare the Data for Training

In [50]:
df = data[data.trap !=''].copy().drop(columns=['max_high_1dlater', 'min_low_1dlater'])
df['trap'] = df['trap'].astype(int)

In [51]:
# Calculate Features
df["EMA_Diff"] = df["ema_fast"] - df["ema_low"]  # Distance between EMAs
df["ATR"] = ta.atr(df["High"], df["Low"], df["Close"], length=14)  # Volatility
df["RSI"] = ta.rsi(df["Close"], length=14)  # Momentum indicator
df["BB_Width"] = ta.bbands(df["Close"], length=20)["BBU_20_2.0"] - ta.bbands(df["Close"], length=20)["BBL_20_2.0"]
df["Volume_Change"] = df["Volume"] / df["Volume"].shift(1)  # Relative volume

## Train a Classification Model

In [52]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [53]:
# Define Features and Target Variable
features = ["EMA_Diff", "ATR", "RSI", "BB_Width", "Volume_Change"]
X = df[features]
y = df["trap"]

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [54]:
y

Date
2018-08-21 14:00:00    0
2018-08-30 10:15:00    1
2018-08-30 11:20:00    0
2018-09-04 10:35:00    0
2018-09-11 13:40:00    0
                      ..
2025-02-04 13:40:00    0
2025-02-10 09:50:00    1
2025-02-12 09:10:00    1
2025-02-13 09:10:00    1
2025-02-13 13:45:00    0
Name: trap, Length: 547, dtype: int64

In [55]:
# Train Random Forest Model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [56]:
# Predictions
y_pred = model.predict(X_test)

In [57]:
# Evaluate Performance
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.6181818181818182
              precision    recall  f1-score   support

           0       0.59      0.42      0.49        48
           1       0.63      0.77      0.70        62

    accuracy                           0.62       110
   macro avg       0.61      0.60      0.59       110
weighted avg       0.61      0.62      0.60       110

